In [36]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

In [47]:
input_type = 'train'
agg_roles_dict={'Role':  lambda x : x.mode() }
df_rol_raw = pd.read_csv('../data/' + input_type + '_roles_raw.csv', low_memory=False)
df_rol_raw.head()

,id,Person.ID,Role
0,1,40572,CHIEF_INVESTIGATOR
1,2,9067,CHIEF_INVESTIGATOR
2,3,5967,CHIEF_INVESTIGATOR
3,4,78782,PRINCIPAL_SUPERVISOR
4,5,13042,CHIEF_INVESTIGATOR


In [48]:
# we generate a dataframe with the uniqe key pairs ( date, person Id).
# this df will be filled and returned as the mod frame
df_rol_mod = df_rol_raw.loc[:,['id', 'Person.ID']].drop_duplicates()


for col, fun in agg_roles_dict.items():
 
    # we apply the aggregation function to the coloumn
    tmp2 = df_rol_raw.groupby(['id', 'Person.ID'])[col].agg(fun)
    
    # in case the aggregation function returns a list and not a single element, we take the first one
    # if there were only NaN for this day and persion ID, an empty np array is returned-> we chagne it to a NaN
    tmp2 = tmp2.apply(lambda x : x[0] if (isinstance(x, np.ndarray) and len(x) > 0 ) else \
               (np.nan if (isinstance(x, np.ndarray) and len(x) == 0) else x  ))
    
    # we merge the series with the dataframe that stores all the outcome
    df_rol_mod = pd.merge(df_rol_mod, tmp2.to_frame(name = col).reset_index(),how='left',on = ['id', 'Person.ID'])

In [68]:
# read all csv data
df_roles = pd.read_csv('../data/' + input_type + '_role_mod.csv', low_memory=False)
df_p_dyn = pd.read_csv('../data/' + input_type + '_person_dyn_mod.csv', low_memory=False,parse_dates=['date'])
df_p_static = pd.read_csv('../data/' + input_type + '_person_static_mod.csv', low_memory=False)
df_ext = pd.read_csv('../data/' + input_type + '_externals_raw.csv', low_memory=False)

# we also need the combination (id,date)
df_dates = pd.read_csv('../data/' + input_type + '.csv', low_memory=False,parse_dates=['date']).loc[:, ['id','date']].drop_duplicates()

In [59]:
df_roles.head()

,id,Person.ID,Role
0,1,40572,CHIEF_INVESTIGATOR
1,2,9067,CHIEF_INVESTIGATOR
2,3,5967,CHIEF_INVESTIGATOR
3,4,78782,PRINCIPAL_SUPERVISOR
4,5,13042,CHIEF_INVESTIGATOR


In [60]:
df_p_dyn.head()

,date,Person.ID,Dept.No.,Faculty.No.,With.PHD,years_in_uni,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
0,2005-08-11,40572,3073.0,31.0,NaN,0.0,0.0,0.0,4.0,2.0,0.0,0.0
1,2005-11-11,9067,2538.0,25.0,1.0,4.0,0.0,0.0,6.0,12.0,2.0,2.0
2,2005-11-14,5967,2923.0,25.0,1.0,2.0,0.0,0.0,0.0,3.0,5.0,2.0
3,2005-11-14,27307,2923.0,25.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2005-11-14,79652,2498.0,25.0,1.0,4.0,0.0,0.0,1.0,3.0,3.0,3.0


In [61]:
df_p_static.head()

,Person.ID,Year.of.Birth,Country.of.Birth,Home.Language
0,357,1960.0,Australia,English
1,407,1955.0,Australia,English
2,417,1940.0,Australia,English
3,462,1960.0,Australia,English
4,522,1965.0,Australia,English


In [67]:
df_dates.head()

,id,date
0,1,2005-08-11
1,2,2005-11-11
2,3,2005-11-14
3,4,2005-11-15
4,5,2005-11-16


In [75]:
persons = df_dates.merge(df_ext, how = 'left', on = ['id']).\
    merge(df_roles, how = 'left', on = ['id']).\
    merge(df_p_static,how = 'left', on = ['Person.ID']).\
    merge(df_p_dyn,how = 'left', on = ['date','Person.ID'])

,id,date,EXTERNAL_ADVISOR,EXT_CHIEF_INVESTIGATOR,STUDRES,STUD_CHIEF_INVESTIGATOR,Person.ID,Role,Year.of.Birth,Country.of.Birth,...,Dept.No.,Faculty.No.,With.PHD,years_in_uni,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
0,1,2005-08-11,NaN,NaN,NaN,NaN,40572,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,...,3073.0,31.0,NaN,0.0,0.0,0.0,4.0,2.0,0.0,0.0
1,2,2005-11-11,NaN,NaN,NaN,NaN,9067,CHIEF_INVESTIGATOR,1960.0,Australia,...,2538.0,25.0,1.0,4.0,0.0,0.0,6.0,12.0,2.0,2.0
2,3,2005-11-14,NaN,3.0,NaN,NaN,5967,CHIEF_INVESTIGATOR,1955.0,Australia,...,2923.0,25.0,1.0,2.0,0.0,0.0,0.0,3.0,5.0,2.0
3,3,2005-11-14,NaN,3.0,NaN,NaN,27307,CHIEF_INVESTIGATOR,1950.0,Australia,...,2923.0,25.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,2005-11-14,NaN,3.0,NaN,NaN,79652,CHIEF_INVESTIGATOR,1950.0,Asia Pacific,...,2498.0,25.0,1.0,4.0,0.0,0.0,1.0,3.0,3.0,3.0
5,3,2005-11-14,NaN,3.0,NaN,NaN,11667,DELEGATED_RESEARCHER,1950.0,Australia,...,2548.0,25.0,NaN,4.0,0.0,0.0,6.0,14.0,12.0,2.0
6,4,2005-11-15,NaN,3.0,NaN,1.0,78782,PRINCIPAL_SUPERVISOR,1955.0,Australia,...,2678.0,25.0,1.0,2.0,0.0,0.0,0.0,3.0,13.0,3.0
7,4,2005-11-15,NaN,3.0,NaN,1.0,55337,CHIEF_INVESTIGATOR,1975.0,Australia,...,2678.0,25.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5,2005-11-16,NaN,NaN,NaN,NaN,13042,CHIEF_INVESTIGATOR,1965.0,Western Europe,...,2153.0,19.0,1.0,2.0,0.0,0.0,3.0,0.0,1.0,0.0
9,6,2005-11-19,NaN,NaN,NaN,NaN,301472,CHIEF_INVESTIGATOR,1950.0,Australia,...,2533.0,25.0,1.0,1.0,2.0,0.0,7.0,27.0,27.0,6.0


In [51]:
df_rol_raw.loc[:,['id', 'Person.ID']].drop_duplicates().shape

(9735, 2)

In [19]:
persons = pd.concat( [df.iloc[:,0],  df.loc[:,'Person.ID.1':'C.15']], axis = 1)

In [153]:
persons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6633 entries, 0 to 6632
Columns: 226 entries, id to C.15
dtypes: float64(159), int64(1), object(66)
memory usage: 11.4+ MB


In [20]:
persons.head()

,id,Person.ID.1,Role.1,Year.of.Birth.1,Country.of.Birth.1,Home.Language.1,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,...,Dept.No..15,Faculty.No..15,With.PHD.15,No..of.Years.in.Uni.at.Time.of.Grant.15,Number.of.Successful.Grant.15,Number.of.Unsuccessful.Grant.15,A..15,A.15,B.15,C.15
0,1,40572.0,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,Other,3073.0,31.0,NaN,Less than 0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9067.0,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,2538.0,25.0,Yes,more than 15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5967.0,CHIEF_INVESTIGATOR,1955.0,Australia,NaN,2923.0,25.0,Yes,>5 to 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,78782.0,PRINCIPAL_SUPERVISOR,1955.0,Australia,NaN,2678.0,25.0,Yes,>5 to 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,13042.0,CHIEF_INVESTIGATOR,1965.0,Western Europe,NaN,2153.0,19.0,Yes,>5 to 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#we flatten the persons subtable
#all columns with persons informatin is repeated 15 times
n_persons = 15

#this gives us the following number of coloumns per person (also 15):
n_persons_cols = (len(persons.columns) - 1) // n_persons

#we generate a list the with ID and the field names
col_names = [persons.columns[0]]
[col_names.append(i[:-2]) for i in persons.columns[1:n_persons_cols+1]]

# and use this list to make an empty dataframe
df_p = pd.DataFrame(columns = col_names)

#we loop over all persons 1..15 and take the respective fields (ie columns) and append them
for i in range(n_persons):
    #coloumn index for first coloumn of person i
    k = 1 + (i*n_persons_cols)
    #coloumn index for last coloumn of person i
    l = 1 + ((i+1)*n_persons_cols)
    #get a temp table with the desired info
    tmp = pd.concat( [persons.iloc[:,0],  persons.iloc[:,k:l]], axis = 1)
    #rename the table to make the UNION ALL opertion work
    tmp.columns = df_p.columns
    df_p = pd.concat([df_p, tmp],axis = 0)



In [22]:
# remove lines with person ID AND Role empty
df_p = df_p.loc[~(df_p.loc[:,'Person.ID'].isnull() & df_p.loc[:,'Role'].isnull()),:]

In [23]:
df_p.head()

,id,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,No..of.Years.in.Uni.at.Time.of.Grant,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
0,1,40572.0,CHIEF_INVESTIGATOR,1965.0,Asia Pacific,Other,3073.0,31.0,NaN,Less than 0,0.0,0.0,4.0,2.0,0.0,0.0
1,2,9067.0,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,2538.0,25.0,Yes,more than 15,0.0,0.0,6.0,12.0,2.0,2.0
2,3,5967.0,CHIEF_INVESTIGATOR,1955.0,Australia,NaN,2923.0,25.0,Yes,>5 to 10,0.0,0.0,0.0,3.0,5.0,2.0
3,4,78782.0,PRINCIPAL_SUPERVISOR,1955.0,Australia,NaN,2678.0,25.0,Yes,>5 to 10,0.0,0.0,0.0,3.0,13.0,3.0
4,5,13042.0,CHIEF_INVESTIGATOR,1965.0,Western Europe,NaN,2153.0,19.0,Yes,>5 to 10,0.0,0.0,3.0,0.0,1.0,0.0


In [24]:
# we create a new table called externals with the  columns grant_application_id and has_[external role]
# external roles are stored as person without ID, therefore the following command identifies them:
# df_p.loc[df_p.loc[:,'Person.ID'].isnull(),'Role'].unique()
# of course they can also be identifed by name (ie external advisor..)
# it might be possilbe that a different role has a missing person ID filed in the testing date,
# therefore we hardcode it: EXT_CHIEF_INVESTIGATOR', 'STUD_CHIEF_INVESTIGATOR', 'STUDRES', 'EXTERNAL_ADVISOR'
ext_list = ['EXT_CHIEF_INVESTIGATOR', 'STUD_CHIEF_INVESTIGATOR', 'STUDRES', 'EXTERNAL_ADVISOR']
mask = df_p.Role.isin(ext_list)
df_ext = df_p.loc[mask,:]
#we only want to keep the grant application id and the role
df_ext = df_ext.loc[:,['id','Role']]

df_p = df_p.loc[~mask,:]

In [25]:
# next step is to pivot the table so that we have one line per grant applications ID
df_ext['dummy'] = 1
df_ext = df_ext.pivot_table(index ='id', columns='Role', values = 'dummy', aggfunc=np.sum )

# we restore the column ID by restting the index
df_ext = df_ext.reset_index()

In [161]:
# we write the externals table to a csv file
df_ext.to_csv( '../data/'+ input_type + '_externals_raw.csv',index = False)


In [26]:
# this should be 0
df_p.loc[:,'Person.ID'].isnull().sum()

0

In [27]:
# we can now continue with the person table
# we make all categorial variables to strings
# numerics are first converted to integers, otherwise we will have .0 at the end
df_p[['Person.ID']] = df_p[['Person.ID']].astype(int).astype(str)
df_p[['Dept.No.','Faculty.No.']] = df_p[['Dept.No.','Faculty.No.']].astype(str)

In [28]:
# next columns to work on is the years spend in the uni
# we rename the column
df_p = df_p.rename(columns={'No..of.Years.in.Uni.at.Time.of.Grant': 'years_in_uni'})
# we use a dict to remap the values
dict_years_in_uni = {'Less than 0': 0.,
'>=0 to 5': 1.,
'>5 to 10': 2.,
'>10 to 15': 3.,
'more than 15': 4.}
df_p = df_p.replace({'years_in_uni' : dict_years_in_uni})
df_p.tail()

,id,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,years_in_uni,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
5815,5816,69732,CHIEF_INVESTIGATOR,1950.0,Australia,NaN,2153.0,19.0,Yes,4.0,4.0,1.0,1.0,39.0,2.0,0.0
4126,4127,74722,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,3048.0,31.0,Yes,3.0,1.0,0.0,13.0,5.0,10.0,3.0
4283,4284,74722,CHIEF_INVESTIGATOR,1960.0,Australia,NaN,3048.0,31.0,Yes,3.0,1.0,0.0,18.0,6.0,11.0,3.0
4126,4127,79977,CHIEF_INVESTIGATOR,1935.0,Australia,NaN,3048.0,31.0,NaN,4.0,1.0,0.0,10.0,3.0,2.0,NaN
4283,4284,79977,CHIEF_INVESTIGATOR,1935.0,Australia,NaN,3048.0,31.0,NaN,4.0,1.0,0.0,13.0,3.0,2.0,NaN


In [29]:
# we split it in two dataframes: static and dynamic
static_columns = ['Person.ID','Year.of.Birth','Country.of.Birth','Home.Language']
df_p_static = df_p.loc[:,static_columns]
df_p_static.to_csv(  '../data/'+ input_type + '_person_static_raw.csv',index = False)



In [30]:
# in the dynamic df comes ecervything that is not in the static + the key
df_p_dyn = df_p.loc[:,( (~(df_p.columns.isin(static_columns))) | df_p.columns.isin(['id','Person.ID']))]
# we only remove the Role, since we will have a seperate table for this
df_p_dyn = df_p_dyn.loc[:,~df_p_dyn.columns.isin(['Role']) ]

#df_p_before_join = df_p_dyn.loc[:,'id'].size

df_p_dyn  = pd.merge( df.loc[:,['id','date']], df_p_dyn, on = 'id')
#print(df_p_dyn.loc[:,'id'].size == df_p_before_join)

df_p_dyn = df_p_dyn.loc[:,~df_p_dyn.columns.isin(['id']) ]

df_p_dyn.to_csv(  '../data/'+ input_type + '_person_dyn_raw.csv',index = False)

In [31]:
#we extract the roles
roles = df_p.loc[:,['id','Person.ID','Roles']]

In [32]:
roles.to_csv(  '../data/'+ input_type + '_roles_raw.csv',index = False)

In [13]:
df_p_dyn.loc[df_p_dyn.loc[:,'Dept.No.'] == 'nan',:] = np.nan
df_p_dyn.loc[df_p_dyn.loc[:,'Dept.No.'] == 'nan',:]

,date,Person.ID,Dept.No.,Faculty.No.,With.PHD,No..of.Years.in.Uni.at.Time.of.Grant,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
17,19/11/05,89462,nan,nan,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0
45,19/11/05,3067,nan,nan,NaN,NaN,0.0,0.0,3.0,3.0,1.0,0.0
46,19/11/05,51027,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
56,19/11/05,537,nan,25.0,Yes,>10 to 15,2.0,0.0,9.0,9.0,13.0,2.0
71,19/11/05,78817,nan,25.0,Yes,more than 15,2.0,0.0,4.0,8.0,6.0,2.0
88,19/11/05,65227,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
90,19/11/05,46237,nan,nan,NaN,NaN,0.0,0.0,0.0,1.0,2.0,1.0
98,19/11/05,41202,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
164,19/11/05,17512,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
178,19/11/05,64782,nan,nan,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [220]:
df_p_dyn.groupby('Person.ID').size().sort_values()

Person.ID
99877      1
7037       1
30582      1
30612      1
3062       1
30632      1
703262     1
70192      1
30777      1
30827      1
700267     1
700037     1
30987      1
30997      1
704062     1
31012      1
699612     1
31237      1
69952      1
313557     1
699007     1
314067     1
69887      1
314557     1
697842     1
314962     1
31512      1
315512     1
69782      1
69977      1
          ..
71867     20
1757      20
76352     21
66887     21
79032     21
78817     22
9067      22
34737     22
69032     22
734322    23
46132     23
84622     24
28607     24
498427    24
73757     24
16137     25
69812     26
72037     28
82567     28
67647     30
67037     31
12957     32
74162     32
36252     33
17342     33
85862     35
67842     37
85642     39
79192     57
1797      72
Length: 2498, dtype: int64

In [55]:
df_p.loc[df_p.Role == 'STUD_CHIEF_INVESTIGATOR','Person.ID'].unique()

array([ nan])

In [52]:
df_p.loc[ df_p.loc[:,'Person.ID'] == 489582,:]

,id,Person.ID,Role,Year.of.Birth,Country.of.Birth,Home.Language,Dept.No.,Faculty.No.,With.PHD,No..of.Years.in.Uni.at.Time.of.Grant,Number.of.Successful.Grant,Number.of.Unsuccessful.Grant,A.,A,B,C
69,70,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,0.0,1.0,11.0,1.0,2.0,0.0
2427,2428,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,1.0,1.0,12.0,1.0,3.0,1.0
3173,3174,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,1.0,12.0,1.0,3.0,1.0
4221,4222,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,1.0,13.0,1.0,3.0,1.0
4828,4829,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,13.0,1.0,3.0,1.0
5493,5494,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,13.0,1.0,3.0,1.0
3154,3155,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,1.0,1.0,0.0,0.0,0.0,0.0
3972,3973,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,1.0,0.0,0.0,0.0,0.0
6183,6184,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,0.0,0.0,0.0,0.0
5725,5726,489582.0,CHIEF_INVESTIGATOR,1975.0,Great Britain,NaN,3048.0,31.0,Yes,>=0 to 5,2.0,2.0,13.0,1.0,3.0,1.0


In [141]:
df.date

0       2005-08-11
1       2005-11-11
2       2005-11-14
3       2005-11-15
4       2005-11-16
5       2005-11-19
6       2005-11-19
7       2005-11-19
8       2005-11-19
9       2005-11-19
10      2005-11-19
11      2005-11-19
12      2005-11-19
13      2005-11-19
14      2005-11-19
15      2005-11-19
16      2005-11-19
17      2005-11-19
18      2005-11-19
19      2005-11-19
20      2005-11-19
21      2005-11-19
22      2005-11-19
23      2005-11-19
24      2005-11-19
25      2005-11-19
26      2005-11-19
27      2005-11-19
28      2005-11-19
29      2005-11-19
           ...    
6603    2007-10-12
6604    2007-11-12
6605    2007-11-12
6606    2007-12-12
6607    2007-12-12
6608    2007-12-13
6609    2007-12-13
6610    2007-12-13
6611    2007-12-14
6612    2007-12-14
6613    2007-12-14
6614    2007-12-14
6615    2007-12-14
6616    2007-12-17
6617    2007-12-17
6618    2007-12-17
6619    2007-12-17
6620    2007-12-17
6621    2007-12-18
6622    2007-12-18
6623    2007-12-18
6624    2007

In [171]:
df_p.loc[:,'Role'].unique()

array(['CHIEF_INVESTIGATOR', 'PRINCIPAL_SUPERVISOR',
       'DELEGATED_RESEARCHER', 'HONVISIT', nan], dtype=object)